In [464]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn import preprocessing 
import math
import xgboost as xgb
import datetime
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import KFold 
from sklearn.metrics import matthews_corrcoef, make_scorer
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [339]:
#reading the training data, kindly change the path according to your file
train_data = pd.read_csv('train_upd.csv') 

In [465]:
#creating the feature : avg_data_speed (Find description in report)
train_data['total_bytes'] = 0
byte_cols = ['web_browsing_total_bytes',
 'video_total_bytes',
 'social_ntwrking_bytes',
 'cloud_computing_total_bytes',
 'web_security_total_bytes',
 'gaming_total_bytes',
 'health_total_bytes',
 'communication_total_bytes',
 'file_sharing_total_bytes',
 'remote_access_total_bytes',
 'photo_sharing_total_bytes',
 'software_dwnld_total_bytes',
 'marketplace_total_bytes',
 'storage_services_total_bytes',
 'audio_total_bytes',
 'location_services_total_bytes',
 'presence_total_bytes',
 'advertisement_total_bytes',
 'system_total_bytes',
 'voip_total_bytes',
 'speedtest_total_bytes',
 'email_total_bytes',
 'weather_total_bytes',
 'media_total_bytes',
 'mms_total_bytes',
 'others_total_bytes']
train_data['speed'] = 0
for i in byte_cols:
    train_data['total_bytes'] += train_data[i]
train_data.speed = (train_data.total_bytes/(train_data.par_min*train_data.subscriber_count))

In [341]:
#encoding the vendor data
train_data.ran_vendor.replace({'NOKIA':1,'ERICSSON':2,'HUAWEI':3},inplace=True)

In [342]:
#creating the new feature : week_day ( find desciption in report)
temp =[]
for i in range(len(train_data)):
    temp.append(datetime.datetime(train_data['par_year'][i],train_data['par_month'][i],train_data['par_day'][i]).weekday())
train_data["week_day"] = temp

In [343]:
#creating the target variable
y = train_data.copy()
y = y["Congestion_Type"]

In [344]:
#because par_year and par_month features were useless we are dropping them
#also par_day feature is already converted into week_day
x1=train_data.drop(['par_year','par_month','par_day'],axis=1)

In [345]:
#dropping the target variable from the data
x = x1.drop(['Congestion_Type'],axis = 1)

In [346]:
#splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [347]:
#dropping the cell name from X_train but we are keeping cell name in x2 for future merging use
x2 = X_train.copy()
X_train.drop(['cell_name'],axis = 1,inplace =True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [348]:
#this is the first step of two step model,here we convert the problem into binary classification by converting
#all the not-NC target rows into 'C'
y2 = y_train.copy()
y_train[y_train == "4G_BACKHAUL_CONGESTION"] = "C"
y_train[y_train == "4G_RAN_CONGESTION"] = "C"
y_train[y_train == "3G_BACKHAUL_CONGESTION"] = "C"

In [349]:
#encoding the y_train into 0,1
le = preprocessing.LabelEncoder()
mvar47 = le.fit_transform(y_train.astype(str))
y_train = mvar47


In [56]:
#this is the xgboost crossvalidation for determining the best learning rate and n_round/n_estimators
#we define a custom metric function mat, which gives us the MCC cross-validation score

threshold = 0.5
def mat(preds, dtrain):
    labels = dtrain.get_label()
    return 'MCC', matthews_corrcoef(labels, preds > threshold)
dtrain = xgb.DMatrix(X_train,label = y_train)

#this gives us the cross-validation search
param = {'max_depth':3, 'eta':0.1,'objective':'binary:logistic','silent':1,'n_thread' :4}

#we create a custom scorer which gives us the cross-validation MCC
my_scorer = make_scorer(matthews_corrcoef)

#the cross-validation process starts here, this code will take time to run
xgbcv = xgb.cv(param,dtrain,num_boost_round=1500,nfold = 4,verbose_eval = 50,feval = mat,maximize = True )

#we found best cross-validation MCC for eta/learning_rate = 0.1 and n_boost_round/n_estimators = 900

[0]	train-MCC:0.486152+0.00698812	train-error:0.183549+0.000245231	test-MCC:0.48318+0.0105985	test-error:0.18462+0.00160011
[50]	train-MCC:0.850405+0.00160637	train-error:0.056215+0.000514387	test-MCC:0.829794+0.00638507	test-error:0.0637888+0.00234391
[100]	train-MCC:0.879619+0.00146734	train-error:0.0458302+0.000534293	test-MCC:0.851305+0.00582669	test-error:0.0564538+0.00214296
[150]	train-MCC:0.889174+0.00160302	train-error:0.0422713+0.000559299	test-MCC:0.85856+0.00594968	test-error:0.0537485+0.00222036
[200]	train-MCC:0.894866+0.00173414	train-error:0.0401075+0.000646861	test-MCC:0.861293+0.0061075	test-error:0.0527622+0.00228978
[250]	train-MCC:0.898211+0.00153135	train-error:0.0388343+0.000567994	test-MCC:0.861258+0.00592105	test-error:0.0527145+0.00216163
[300]	train-MCC:0.900922+0.00153287	train-error:0.0377897+0.000574558	test-MCC:0.861942+0.0057464	test-error:0.0524437+0.00213889
[350]	train-MCC:0.903466+0.00150801	train-error:0.0368033+0.000575655	test-MCC:0.8616+0.0058426

KeyboardInterrupt: 

In [131]:

my_scorer = make_scorer(matthews_corrcoef)

#now we do grid search for tuning parameters other than eta and n_round namely subsample,colsample_bytree,max_depth,min_child_weight

#this is our paramter grid for the grid search
param_test1 = {'subsample':[0.2,0.4,0.6,0.8,1],'colsample_bytree':[0.2,0.4,0.6,0.8,1],'max_depth':[2,3,4,5,6,7],'min_child_weight' : [2,3,4,5,6,7]}

#this is the grid search function,it will provide us the best cross-validation MCC for the grid. It will take time to run
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=900,gamma=0,objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
param_grid = param_test1, scoring=my_scorer,n_jobs=4,iid=False, cv=4,verbose = 10)
gsearch1.fit(X_train,y_train)

#we found the best parameters to be colsample_bytree=1 ,min_child_weight=4, subsample=1, max_depth=2
#Cross validation MCC for model1 =  86.39

Fitting 4 folds for each of 25 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:  3.5min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  4.2min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  6.7min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  8.3min
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed: 11.2min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed: 13.0min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed: 17.3min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed: 20.7min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed: 23.2min finished


GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=4, missing=None, n_estimators=900,
       n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=1),
       fit_params=None, iid=False, n_jobs=4,
       param_grid={'subsample': [0.2, 0.4, 0.6, 0.8, 1], 'colsample_bytree': [0.2, 0.4, 0.6, 0.8, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(matthews_corrcoef), verbose=10)

In [201]:
#prints the best params
gsearch1.best_params_

NameError: name 'gsearch1' is not defined

In [350]:
#fitting an Xgboost model with the best found parameters
xgb1 = xgb.XGBClassifier(booster='gbtree', colsample_bylevel=1,
    colsample_bytree=1, gamma=0, learning_rate=0.1,
    max_delta_step=0, max_depth=2, min_child_weight=4, missing=None,
    n_estimators=900, n_jobs=3, nthread=1, objective='multi:softmax',
    random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
    seed=None, subsample=1, num_class = 2)

In [351]:
#fitting model on training data
xgb1.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=4, missing=None, n_estimators=900,
       n_jobs=3, nthread=1, num_class=2, objective='multi:softmax',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [353]:
#creating new data with only the congestion target for training
new_data = x2[y2!= 'NC']

In [354]:
#creating only congestion target variable
y2 = y2[y2!= "NC"]

In [355]:
#encoding into 0,1,2
le = preprocessing.LabelEncoder()
mvar47 = le.fit_transform(y2.astype(str))
y2 = mvar47


In [ ]:
#tuning the eta and n_round for 2nd model like the 1st one
def mat(preds, dtrain):
    labels = dtrain.get_label()
    return 'MCC', matthews_corrcoef(labels, preds)
dtrain = xgb.DMatrix(new_data.loc[:, new_data.columns != 'cell_name'],label = y2)


param = {'max_depth':4, 'eta':0.1,'objective':'multi:softmax','silent':1,'n_thread' :4,'num_class':3}
my_scorer = make_scorer(matthews_corrcoef)
xgbcv = xgb.cv(param,dtrain,num_boost_round=850,nfold = 4,verbose_eval = 50,feval = mat,maximize = True )

#we found the best eta = 0.1 and n_boost_round = 850

[0]	train-MCC:0.365691+0.00475269	train-merror:0.428314+0.00432515	test-MCC:0.346448+0.00834121	test-merror:0.441004+0.00704769
[50]	train-MCC:0.72599+0.00162118	train-merror:0.183673+0.00114577	test-MCC:0.651312+0.00457876	test-merror:0.233454+0.0030945
[100]	train-MCC:0.766429+0.00135166	train-merror:0.156026+0.000910178	test-MCC:0.679774+0.00478287	test-merror:0.213696+0.00325228
[150]	train-MCC:0.785286+0.000801388	train-merror:0.143421+0.000548677	test-MCC:0.689474+0.00418408	test-merror:0.207167+0.00282601
[200]	train-MCC:0.797419+0.00118776	train-merror:0.135318+0.000806677	test-MCC:0.694093+0.00587931	test-merror:0.204084+0.00395866
[250]	train-MCC:0.807342+0.00168688	train-merror:0.128697+0.00114088	test-MCC:0.696611+0.00583214	test-merror:0.202403+0.00392799
[300]	train-MCC:0.816658+0.00208667	train-merror:0.12248+0.00140676	test-MCC:0.696975+0.00616976	test-merror:0.202148+0.00414608
[350]	train-MCC:0.824046+0.00158547	train-merror:0.117546+0.00107845	test-MCC:0.697704+0.005

In [ ]:
#setting up the tuning grid
my_scorer = make_scorer(matthews_corrcoef)

param_test1 = {'subsample':[0.2,0.4,0.6,0.8,1],'colsample_bytree':[0.2,0.4,0.6,0.8,1],'max_depth':[2,3,4,5,6,7],'min_child_weight' : [2,3,4,5,6,7]}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=850,gamma=0, max_depth=2, min_child_weight=4,
objective= 'multi:softmax', nthread=4, scale_pos_weight=1, seed=27), 
param_grid = param_test1, scoring=my_scorer,n_jobs=4,iid=False, cv=4,verbose = 10)
gsearch1.fit(new_data.loc[:, new_data.columns != 'cell_name'],y2)

#we found the best parameters to be colsample_bytree=0.2 ,min_child_weight=4, subsample=0.6, max_depth=2
#Cross validation MCC for model2 =  70.44

Fitting 4 folds for each of 25 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  8.7min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 12.2min
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed: 16.8min
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 19.5min
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 26.4min
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 30.0min


In [356]:
#This is the second model which is capable of predicting 0,1,2 for different congestion types (3g,4g-ran and 4g-backhaul)
xgb2 = xgb.XGBClassifier(booster='gbtree', colsample_bylevel=1,
    colsample_bytree=0.2, gamma=0, learning_rate=0.1,
    max_delta_step=0, max_depth=2, min_child_weight=4, missing=None,
    n_estimators=850, n_jobs=3, nthread=1, objective='multi:softmax',
    random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
    seed=None, subsample=0.6)

In [357]:
#fitting on training data for second model
xgb2.fit(new_data.loc[:, new_data.columns != 'cell_name'],y2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.2, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=2, min_child_weight=4, missing=None, n_estimators=850,
       n_jobs=3, nthread=1, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.6)

In [359]:
#encoding the 20% target variable(testing)
le = preprocessing.LabelEncoder()
mvar47 = le.fit_transform(y_test.astype(str))
y_test = mvar47

In [360]:
#predicting the C vs NC on test data using model1
pred1_test = xgb1.predict(X_test.loc[:, X_test.columns != 'cell_name'])

In [361]:
#list of cell names with congestion = NC
l = X_test[pred1_test == 1]['cell_name'].tolist()

In [363]:
#creating the dataframe of cell_name of NCs and Predictions which is equal to 3 for NC (using inverse tranformation of encoder)
df = pd.DataFrame(l)
df['cell_name'] = df[0]
df.head()
df.drop(0,inplace = True,axis =1)
df['Predictions'] = 3
dfnew = pd.merge(X_test,df)

In [364]:
#list of cellname with congestion = C
l2 =X_test[pred1_test == 0]['cell_name'].tolist()

In [365]:
#new testing data with only congestion = C
new_test = X_test[pred1_test == 0]
new_test1 = new_test.drop(['cell_name'],axis=1)

In [366]:
#predicting on the data with congestion = C to further classify it into 3g,4g-ran,4g-backhaul
pred2_test = xgb2.predict(new_test1)

In [367]:
#using encoder inverse transform we know that 3g : 0 , 4g_backhaul : 1, 4g_ran :2 , NC : 3
new_test['Predictions'] = 10000 # initializing the column
new_test['Predictions'][pred2_test == 0] = 0
new_test['Predictions'][pred2_test == 1] = 1
new_test['Predictions'][pred2_test == 2] = 2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(

In [369]:
#concating the two dataframes which have predictions for NC(dfnew) and different classes of C(new_test1)
iopp = pd.concat([dfnew,new_test])

In [370]:
#sorting by cell name
iopp = iopp.sort_values(by = 'cell_name')

In [373]:
y_test1 = y_test1.sort_values(by = 'cell_name')

In [374]:
y_test1.head()

,cell_name,4G_rat,par_hour,par_min,subscriber_count,web_browsing_total_bytes,video_total_bytes,social_ntwrking_bytes,cloud_computing_total_bytes,web_security_total_bytes,...,mms_total_bytes,others_total_bytes,beam_direction,cell_range,tilt,ran_vendor,total_bytes,total_bytes_mod,week_day,True_Values
60240,260006,1,4,30,1128,1009,206,602,414,36,...,66,172,74,6,4,3,4279,0.0,6,0
38308,395468,0,11,35,823,68220,31059,4187,238,3,...,11,165,119,5,2,3,110272,0.0,4,1
19851,412780,0,6,60,609,62,1491,1837,36,8,...,5,37,117,6,3,1,12133,0.0,2,0
28602,442738,1,10,35,487,93,432,12760,182,23,...,18,95,110,2,2,2,19402,0.0,4,3
42517,476133,0,2,55,8,1291,1973,9308,514,58,...,37,7,79,3,2,2,22142,0.0,0,1


In [375]:
iopp.head()

,cell_name,4G_rat,par_hour,par_min,subscriber_count,web_browsing_total_bytes,video_total_bytes,social_ntwrking_bytes,cloud_computing_total_bytes,web_security_total_bytes,...,mms_total_bytes,others_total_bytes,beam_direction,cell_range,tilt,ran_vendor,total_bytes,total_bytes_mod,week_day,Predictions
60240,260006,1,4,30,1128,1009,206,602,414,36,...,66,172,74,6,4,3,4279,0.0,6,0
38308,395468,0,11,35,823,68220,31059,4187,238,3,...,11,165,119,5,2,3,110272,0.0,4,1
3925,412780,0,6,60,609,62,1491,1837,36,8,...,5,37,117,6,3,1,12133,0.0,2,3
3097,442738,1,10,35,487,93,432,12760,182,23,...,18,95,110,2,2,2,19402,0.0,4,3
42517,476133,0,2,55,8,1291,1973,9308,514,58,...,37,7,79,3,2,2,22142,0.0,0,0


In [376]:
#getting the matthews coef:
matthews_corrcoef(y_test1['True_Values'],iopp['Predictions'])

#by running all this code 5 times with different training and testing data(we did not fix random state while splitting)
#we got the average MCC to be 0.7395

0.7376531938076256

In [423]:
#reading the final testing file we have to predict
finaltest = pd.read_csv('test_upd.csv')

In [424]:
#creating respective features which were made in training and other preprocessing
finaltest['total_bytes'] = 0
byte_cols = ['web_browsing_total_bytes',
 'video_total_bytes',
 'social_ntwrking_bytes',
 'cloud_computing_total_bytes',
 'web_security_total_bytes',
 'gaming_total_bytes',
 'health_total_bytes',
 'communication_total_bytes',
 'file_sharing_total_bytes',
 'remote_access_total_bytes',
 'photo_sharing_total_bytes',
 'software_dwnld_total_bytes',
 'marketplace_total_bytes',
 'storage_services_total_bytes',
 'audio_total_bytes',
 'location_services_total_bytes',
 'presence_total_bytes',
 'advertisement_total_bytes',
 'system_total_bytes',
 'voip_total_bytes',
 'speedtest_total_bytes',
 'email_total_bytes',
 'weather_total_bytes',
 'media_total_bytes',
 'mms_total_bytes',
 'others_total_bytes']
finaltest['total_bytes_mod'] = 0
for i in byte_cols:
    finaltest['total_bytes'] += finaltest[i]
finaltest.total_bytes_mod = (finaltest.total_bytes_mod/(finaltest.par_min*finaltest.subscriber_count))

In [425]:
finaltest.ran_vendor.replace({'NOKIA':1,'ERICSSON':2,'HUAWEI':3},inplace=True)
import datetime
temp =[]
for i in range(len(finaltest)):
    temp.append(datetime.datetime(finaltest['par_year'][i],finaltest['par_month'][i],finaltest['par_day'][i]).weekday())
finaltest["week_day"] = temp

In [426]:
finaltest = finaltest.drop(['par_year','par_month','par_day'],axis=1)

In [427]:
#predicting the C vs NC using model1
finalpred1 = xgb1.predict(finaltest.loc[:, finaltest.columns != 'cell_name'])

In [431]:
#list of cellnames with NC
lfinal = finaltest[finalpred1 == 1]['cell_name'].tolist()

In [432]:
#similar to inhouse testing
dffinal = pd.DataFrame(lfinal)
dffinal['cell_name'] = dffinal[0]
dffinal.head()
dffinal.drop(0,inplace = True,axis =1)
dffinal['Predictions'] = 3
dffinalnew = pd.merge(finaltest,dffinal)

In [433]:
#list of cellnames with C
l2final = finaltest[finalpred1 == 0]['cell_name'].tolist()

In [434]:
#taking only that data which our model1 predicted to be a congestion
new_test_final = finaltest[finalpred1 == 0]
new_test1_final = new_test_final.drop(['cell_name'],axis=1)

In [435]:
#predicting the congestion type
finalpred2 = xgb2.predict(new_test1_final)

In [438]:
#adding encoded variable to a new dataframe
new_test_past['Predictions'] = 10000
new_test_past['Predictions'][past_pred2 == 0] = 0
new_test_past['Predictions'][past_pred2 == 1] = 1
new_test_past['Predictions'][past_pred2 == 2] = 2

6262

In [441]:
#concating the predictions for both C vs NC and different types of C
iopp = pd.concat([dffinalnew,new_test_final])

In [445]:
#creating the final submission file
submission = pd.merge(finaltest,iopp[['cell_name','Predictions']],on = 'cell_name')

In [451]:
submission = submission[['cell_name','Predictions']]

In [454]:
submission['Congestion_Type'] = submission['Predictions']
submission.drop('Predictions',axis =1 ,inplace = True)

In [459]:
#inverse transforming the labeled data
submission['Congestion_Type'] = le.inverse_transform(submission['Congestion_Type'])

In [463]:
#writing to csv
submission.to_csv('submission.csv',index = False)

In [ ]:
#the next few cells will deal with the plots we created for the report

In [470]:
#this is the plot for xgboost cross-validation run to tune eta and n_round

train_MCC = [0.850405,0.879619,0.889174,0.894866,0.898211,0.900922,0.903466,0.906189,0.908533,0.910864]
test_MCC = [0.829794,0.851305,0.85856,0.861293,0.861258,0.861942,0.8616,0.863502,0.862797,0.863356]
nround = [50,100,150,200,250,300,350,400,450,500]
n_chart = pd.DataFrame(nround)
n_chart.columns = ['n_round']
n_chart['train_MCC'] = train_MCC
n_chart['test_MCC'] = test_MCC
n_chart.head()

,n_round,train_MCC,test_MCC
0,50,0.850405,0.829794
1,100,0.879619,0.851305
2,150,0.889174,0.858560
3,200,0.894866,0.861293
4,250,0.898211,0.861258


In [471]:
fig, ax = plt.subplots(figsize=(10,5))
plt.plot(n_chart['n_round'],n_chart['train_MCC'])
fig, ax = plt.subplots(figsize=(10,5))
plt.plot(n_chart['n_round'],n_chart['test_MCC'])
plt.savefig('tuning.jpg')

In [ ]:
#this is the code to generate correlation plot

In [469]:
x_name = x.copy()
corr = x_name.corr()
fig, ax = plt.subplots(figsize=(20,10))
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.savefig('corr_plot.jpg')

#Please check the foler you're running this script in to find the chart

In [ ]:
#next one is the xgboost feature importance plot

In [472]:
imp = xgb1.feature_importances_
imp = imp*1000

col_names = ['cell_name' , 'par_hour', 'par_min', 'subscriber_count', 'web_browsing_total_bytes', 'video_total_bytes', 'social_ntwrking_bytes', 'cloud_computing_total_bytes', 'web_security_total_bytes', 'gaming_total_bytes', 'health_total_bytes', 'communication_total_bytes', 'file_sharing_total_bytes', 'remote_access_total_bytes', 'photo_sharing_total_bytes', 'software_dwnld_total_bytes', 'marketplace_total_bytes', 'storage_services_total_bytes', 'audio_total_bytes', 'location_services_total_bytes', 'presence_total_bytes', 'advertisement_total_bytes', 'system_total_bytes', 'voip_total_bytes', 'speedtest_total_bytes', 'email_total_bytes', 'weather_total_bytes', 'media_total_bytes', 'mms_total_bytes', 'others_total_bytes', 'beam_direction', 'cell_range', 'tilt', 'total_bytes', 'speed', 'week_day', 'service']
chart = pd.DataFrame(col_names)
chart['imp'] = imp
chart.columns = ['Feature_Name','Feature_Importance']

chart = chart.sort_values(by=['Feature_Importance'], ascending=False)

fig, ax = plt.subplots(figsize=(20,10))
plt.title('Feature importance plot')
plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
ax = sns.barplot(x=chart['Feature_Importance'], y=chart['Feature_Name'])
plt.savefig('Final_plot.jpg')
